- Loading 4 million Amazon about 4.51 GB RAM
- Loading 4 million Amazon + 4 million Goodreads about 8.39 GB RAM (sometimes 10 GB?)
- Creating 8 million combined DF with ASIN, Goodreads book ID, user_id, review_text, rating, isAmazon about 19.77 GB RAM
- Loading 8 million combined DF aboout 7.09 GB RAM

Notes
- Load entire Amazon and Goodreads review datasets through batching.
    - Save first 4 million, next 4 million etc. as separate CSVs in Google Drive, then
    load these CSVs as DataFrames and combine the DataFrames into one DF.
    Finally save the DF as a CSV in Drive to make loading time faster.
- Combine Amazon and Goodreads DFs into one dataset. Save as CSV in Drive to make loading time faster. 
Use ISBN and extra Kaggle dataset.
- Split combined dataset DF into train and test. 
Test should only be reviews from Amazon since want to recommend to Amazon users.
- Do ex. item-based CF using ratings, ex. 5 stars, to recommend books. Use
  as baseline result.
    - Display title and authors of final list of recommended books.
- Advanced algorithms, ex. sentiment analysis, ex. find similar users based on text reviews, to improve results.

In [ ]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

import numpy as np

In [ ]:
# install to use Goodreads API
!pip install goodreads_api_client

     |████████████████████████████████| 92kB 4.6MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 133kB 12.9MB/s 
  Created wheel for rauth: filename=rauth-0.7.3-cp36-none-any.whl size=16055 sha256=59d6e444c40c96c8751b1b0bac822bd96fe128f974ccb69ebfb32faefc472e68
  Stored in directory: /root/.cache/pip/wheels/7b/94/5d/81afc278dd5da884a0002563dc4b0fe85f9067a5a40f76f858
Successfully built rauth
ERROR: tensorflow-datasets 4.0.1 has requirement requests>=2.19.0, but you'll have requests 2.18.3 which is incompatible.
ERROR: tensorboard 2.3.0 has requirement requests<3,>=2.21.0, but you'll have requests 2.18.3 which is incompatible.
ERROR: pandas-datareader 0.9.0 has requirement requests>=2.19.0, but you'll have requests 2.18.3 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.18.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import goodreads_api_client as gr

Goodreads API Client is a Python wrapper around the Goodreads API.

https://pypi.org/project/goodreads-api-client/

In [ ]:
  # Set client API key
  api_key = 'afbTRMOw7ZbCHlQS4kDRQ'
  client = gr.Client(developer_key=api_key)

# **Preprocessing**

**Preprocessing** - Amazon

Amazon dataset: https://nijianmo.github.io/amazon/index.html

First, get the Amazon dataset `Books_5.json.gz`, either by running `wget `or using the existing file in Google Drive.

Currently using file in Google Drive.

In [ ]:
# Already done, commented out
# Download the Amazon 5-core Books review subset
# Note: The dataset is 6.61 GB and will take a long time to load (> 5 minutes).
# !wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Books_5.json.gz

In [ ]:
# mount to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# go to folder containing Books_5.json.gz
%cd "/content/drive/My Drive/data/ce256/project"

/content/drive/.shortcut-targets-by-id/1HAaJ2jKojgPKa8wfJoWiFU_cYq8hnytg/project


We can get a Pandas DataFrame from the dataset. Google Colab has limited RAM and will crash if the entire dataset is loaded, so the DataFrame can contain the first 8 million reviews from the dataset.

The DataFrame is combined from separate DataFrames. One has the first 4 million reviews, the other has the next 4 million.

Get the separate DFs by loading the data to a list and converting the list to a DF. 

Then combine the DFs together. The resulting DataFrame is already saved as a CSV in Google Drive.

Currently using the CSV in Google Drive.

Load the metadata as JSON and DataFrame.

From colab notebook: https://colab.research.google.com/drive/1Zv6MARGQcrBbLHyjPVVMZVnRWsRnVMpV


In [ ]:
# Already done, commented out
'''
### load the meta data

# NOTE: This step takes a very long time (> 10 minutes).
# If trying to load all the data,
# Google Colab will crash with a "using all available RAM" error.
data = []
# Get the first 4 million reviews
# 0
# 4000000
# Get the next 4 - 8 million reviews
# start_review_index = 4000000
# max_reviews = 8000000
# Get the next 8 - 12 million reviews
# start_review_index = 8000000
# max_reviews = 12000000
# Get the next 12 - 16 million reviews
# start_review_index = 12000000
# max_reviews = 16000000
# Get the next 16 - 20 million reviews
# start_review_index = 16000000
# max_reviews = 20000000
# Get the next 20 - 24 million reviews
# Index of first review
# start_review_index = 20000000
# Index of last review + 1
# max_reviews = 24000000
# Get the next 24 million reviews and up
# Index of first review
start_review_index = 24000000
# no max_reviews because getting all other reviews including last review
f = gzip.open('Books_5.json.gz')
for index, line in enumerate(f):
    # if (index >= start_review_index and index < max_reviews):
    if (index >= start_review_index):
      # Each JSON line in the file is a review.
      # Convert line of JSON in file to Python dictionary, then append to data list
      review = json.loads(line.strip())
      data.append(review)
f.close()
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])
'''

'\n### load the meta data\n\n# NOTE: This step takes a very long time (> 10 minutes).\n# If trying to load all the data,\n# Google Colab will crash with a "using all available RAM" error.\ndata = []\n# Get the first 4 million reviews\n# 0\n# 4000000\n# Get the next 4 - 8 million reviews\n# start_review_index = 4000000\n# max_reviews = 8000000\n# Get the next 8 - 12 million reviews\n# start_review_index = 8000000\n# max_reviews = 12000000\n# Get the next 12 - 16 million reviews\n# start_review_index = 12000000\n# max_reviews = 16000000\n# Get the next 16 - 20 million reviews\n# start_review_index = 16000000\n# max_reviews = 20000000\n# Get the next 20 - 24 million reviews\n# Index of first review\n# start_review_index = 20000000\n# Index of last review + 1\n# max_reviews = 24000000\n# Get the next 24 million reviews and up\n# Index of first review\nstart_review_index = 24000000\n# no max_reviews because getting all other reviews including last review\nf = gzip.open(\'Books_5.json.gz\')\

In [ ]:
# Already done, commented out
'''
# convert list into pandas dataframe
df = pd.DataFrame.from_dict(data)

print(len(df))
'''

'\n# convert list into pandas dataframe\ndf = pd.DataFrame.from_dict(data)\n\nprint(len(df))\n'

In [ ]:
# Already done, commented out

# Save the first 4 million reviews in amazon_reviews_1.csv
# df.to_csv("amazon_reviews_1", index=False)
# Save the next 4 - 8 million reviews in amazon_reviews_2.csv
# df.to_csv("amazon_reviews_2", index=False)
# Save the next 8 - 12 million reviews in amazon_reviews_3.csv
# df.to_csv("amazon_reviews_3", index=False)
# Save the next 12 - 16 million reviews in amazon_reviews_4.csv
# df.to_csv("amazon_reviews_4", index=False)
# Save the next 16 - 20 million reviews in amazon_reviews_5.csv
# df.to_csv("amazon_reviews_5", index=False)
# Save the next 20 - 24 million reviews in amazon_reviews_6.csv
# df.to_csv("amazon_reviews_6", index=False)
# Save the next 24 million reviews and up in amazon_reviews_7.csv
# df.to_csv("amazon_reviews_7", index=False)
print("Finished saving.")

Finished saving.


Combine the DataFrames containing parts of the Amazon dataset into a combined DataFrame, then save that as a CSV.

Link: https://www.datacamp.com/community/tutorials/joining-dataframes-pandas

In [ ]:
# Already done, commented out
'''
# Go to folder containing amazon_reviews.csv
%cd "/content/drive/My Drive/data/ce256/project"
amazon_0_to_4mill_df = pd.read_csv('amazon_reviews_1')
amazon_4mill_to_8mill_df = pd.read_csv('amazon_reviews_2')
'''

'\n# Go to folder containing amazon_reviews.csv\n%cd "/content/drive/My Drive/data/ce256/project"\namazon_0_to_4mill_df = pd.read_csv(\'amazon_reviews_1\')\namazon_4mill_to_8mill_df = pd.read_csv(\'amazon_reviews_2\')\n'

In [ ]:
# Already done, commented out
'''
# Concatenate the first 8 million review dfs into a combined df. Ignore the index of the separate dfs.
frames = [amazon_0_to_4mill_df, amazon_4mill_to_8mill_df]
combined_amazon_0_to_8mill_df = pd.concat(frames, ignore_index=True)
'''

'\n# Concatenate the first 8 million review dfs into a combined df. Ignore the index of the separate dfs.\nframes = [amazon_0_to_4mill_df, amazon_4mill_to_8mill_df]\ncombined_amazon_0_to_8mill_df = pd.concat(frames, ignore_index=True)\n'

In [ ]:
# Already done, commented out
'''
# Save the combined dataframe as a CSV
# combined_amazon_0_to_8mill_df.to_csv("combined_amazon_reviews_0_to_8mill.csv", index=False)
print("Finished saving.")
'''

'\n# Save the combined dataframe as a CSV\n# combined_amazon_0_to_8mill_df.to_csv("combined_amazon_reviews_0_to_8mill.csv", index=False)\nprint("Finished saving.")\n'

Finished combining the DataFrames containing parts of the Amazon dataset into a combined DataFrame.

Can now use the DataFrame or get it from the CSV.

In [ ]:
# mount to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Get the dataframe from CSV named combined_amazon_reviews_0_to_8mill.csv in Google Drive

# Go to folder containing CSV
%cd "/content/drive/My Drive/data/ce256/project"
# amazon_df = pd.read_csv('combined_amazon_reviews_0_to_8mill.csv')
# load first 4 million reviews
amazon_df = pd.read_csv("amazon_reviews_1")

# Check the CSV in Google Drive has the correct data
amazon_df.head()

/content/drive/My Drive/data/ce256/project


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,False,"03 30, 2005",A1REUF3A1YCPHM,0001713353,{'Format:': ' Hardcover'},TW Ervin II,"The King, the Mice and the Cheese by Nancy Gur...",A story children will love and learn from,1112140800,NaN,NaN
1,5.0,True,"06 20, 2016",AVP0HXC9FG790,0001713353,NaN,Amazon Customer,The kids loved it!,Five Stars,1466380800,NaN,NaN
2,5.0,True,"01 24, 2016",A324TTUBKTN73A,0001713353,{'Format:': ' Paperback'},Tekla Borner,My students (3 & 4 year olds) loved this book!...,Five Stars,1453593600,NaN,NaN
3,5.0,False,"07 9, 2015",A2RE7WG349NV5D,0001713353,{'Format:': ' Paperback'},Deborah K Woroniecki,LOVE IT,Five Stars,1436400000,NaN,NaN
4,5.0,True,"01 18, 2015",A32B7QIUDQCD0E,0001713353,NaN,E,Great!,Five Stars,1421539200,NaN,NaN


Now that we have the Amazon DataFrame, we need to preprocess it before combining it with the GoodReads DataFrame. The ML model can use one combined dataset, which is made up of the two DataFrames.

There should be a way to know which reviews in the combined dataset are from Amazon. This is to make sure the test data are all from Amazon, since we want to recommend books to Amazon users.

To check if a review if from Amazon or Goodreads, we can add an extra column `isAmazon` to each DataFrame. The value is 1 for all Amazon reviews and 0 for all GoodReads reviews.

In [ ]:
# Get the numer of rows of the df. Each row is a review.
num_rows = len(amazon_df.index)
# Create a list of 1's since all reviews in Amazon dataset are from Amazon
list_of_ones = [1] * num_rows
# Add a new column named 'isAmazon' set to the list
amazon_df['isAmazon'] = list_of_ones

In [ ]:
# Check column names
print(list(amazon_df.columns))
# Check number of columns
print(len(list(amazon_df.columns)))

['overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote', 'image', 'isAmazon']
13


**Preprocessing** - Goodreads

Goodreads dataset: https://sites.google.com/eng.ucsd.edu/ucsdbookgraph/home

Get the reviews dataset `goodreads_reviews_dedup.json.gz`, using the existing file in Google Drive.

In [ ]:
# mount to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# go to folder containing goodreads_reviews_dedup.json.gz
%cd "/content/drive/My Drive/data/ce256/project"

/content/drive/.shortcut-targets-by-id/1HAaJ2jKojgPKa8wfJoWiFU_cYq8hnytg/project


Load the reviews.

Notebook: https://github.com/MengtingWan/goodreads/blob/master/samples.ipynb

In [ ]:
# This function shows how to load datasets
def load_data(file_name, head = 4000000): # 500
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1
            data.append(d)
            
            # break if reaches the 100th line
            if (head is not None) and (count > head):
                break
    return data

In [ ]:
# Already done, commented out
'''
# Load and display sample records of book reviews
reviews = load_data('goodreads_reviews_dedup.json.gz')
np.random.choice(reviews)
'''

{'book_id': '22668737',
 'date_added': 'Wed Apr 01 16:27:05 -0700 2015',
 'date_updated': 'Mon Mar 07 21:47:19 -0800 2016',
 'n_comments': 0,
 'n_votes': 2,
 'rating': 3,
 'read_at': 'Sat Mar 05 00:00:00 -0800 2016',
 'review_id': 'b4a2eda146403d57962e0d21234602c3',
 'review_text': '3 Pamphlet loving stars! \n I had downloaded Against the Ropes to review because I quite like the writing of Jeanette Murray but I didn\'t realize at the time it was part 2 of a series. Which made me have to read this book first, so I could get a grip on the story. These books need to be read as a series NOT as a standalones. \n That being said once I started Below the Belt it took me forever to get hooked. I kept putting the book down and starting a new book instead, but I really wanted to get to Graham\'s story in Fight to the Finish, so I had to get through these two books first. \n This series starts with a group of Marine\'s coming together to have the chance at the All Military Games boxing team. At f

In [ ]:
# Already done, commented out
'''
# convert list into pandas dataframe
goodreads_df = pd.DataFrame.from_dict(reviews)

print(len(goodreads_df))
goodreads_df.head()
'''

4000001


,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,24375664,5cd416f3efc3f944fce4ce2db2290d5e,5,Mind blowingly cool. Best science fiction I've...,Fri Aug 25 13:55:02 -0700 2017,Mon Oct 09 08:55:59 -0700 2017,Sat Oct 07 00:00:00 -0700 2017,Sat Aug 26 00:00:00 -0700 2017,16,0
1,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
2,8842281e1d1347389f2ab93d60773d4d,6392944,5e212a62bced17b4dbe41150e5bb9037,3,I haven't read a fun mystery book in a while a...,Mon Jul 24 02:48:17 -0700 2017,Sun Jul 30 09:28:03 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,Mon Jul 24 00:00:00 -0700 2017,6,0
3,8842281e1d1347389f2ab93d60773d4d,22078596,fdd13cad0695656be99828cd75d6eb73,4,"Fun, fast paced, and disturbing tale of murder...",Mon Jul 24 02:33:09 -0700 2017,Sun Jul 30 10:23:54 -0700 2017,Sun Jul 30 15:42:05 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,22,4
4,8842281e1d1347389f2ab93d60773d4d,6644782,bd0df91c9d918c0e433b9ab3a9a5c451,4,A fun book that gives you a sense of living in...,Mon Jul 24 02:28:14 -0700 2017,Thu Aug 24 00:07:20 -0700 2017,Sat Aug 05 00:00:00 -0700 2017,Sun Jul 30 00:00:00 -0700 2017,8,0


In [ ]:
# Save the dataframe as a CSV
# goodreads_df.to_csv("goodreads_reviews_1.csv", index=False)
print("Finished saving.")

Finished saving.


In [ ]:
# Get the dataframe from CSV named goodread_reviews_1.csv in Google Drive

# Go to folder containing CSV
%cd "/content/drive/My Drive/data/ce256/project"
# load first 4 million reviews
goodreads_df = pd.read_csv("goodreads_reviews_1.csv")

# Check the CSV in Google Drive has the correct data
goodreads_df.head()

/content/drive/.shortcut-targets-by-id/1HAaJ2jKojgPKa8wfJoWiFU_cYq8hnytg/project


,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,24375664,5cd416f3efc3f944fce4ce2db2290d5e,5,Mind blowingly cool. Best science fiction I've...,Fri Aug 25 13:55:02 -0700 2017,Mon Oct 09 08:55:59 -0700 2017,Sat Oct 07 00:00:00 -0700 2017,Sat Aug 26 00:00:00 -0700 2017,16,0
1,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
2,8842281e1d1347389f2ab93d60773d4d,6392944,5e212a62bced17b4dbe41150e5bb9037,3,I haven't read a fun mystery book in a while a...,Mon Jul 24 02:48:17 -0700 2017,Sun Jul 30 09:28:03 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,Mon Jul 24 00:00:00 -0700 2017,6,0
3,8842281e1d1347389f2ab93d60773d4d,22078596,fdd13cad0695656be99828cd75d6eb73,4,"Fun, fast paced, and disturbing tale of murder...",Mon Jul 24 02:33:09 -0700 2017,Sun Jul 30 10:23:54 -0700 2017,Sun Jul 30 15:42:05 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,22,4
4,8842281e1d1347389f2ab93d60773d4d,6644782,bd0df91c9d918c0e433b9ab3a9a5c451,4,A fun book that gives you a sense of living in...,Mon Jul 24 02:28:14 -0700 2017,Thu Aug 24 00:07:20 -0700 2017,Sat Aug 05 00:00:00 -0700 2017,Sun Jul 30 00:00:00 -0700 2017,8,0


To check if a review if from Amazon or Goodreads, we can add an extra column `isAmazon` to each DataFrame. The value is 1 for all Amazon reviews and 0 for all GoodReads reviews.

In [ ]:
# Get the numer of rows of the df. Each row is a review.
num_rows = len(goodreads_df.index)
# Create a list of 0's since all reviews in Goodreads dataset are not from Amazon
list_of_zeros = [0] * num_rows
# Add a new column named 'isAmazon' set to the list
goodreads_df['isAmazon'] = list_of_zeros

In [ ]:
# Check column names
print(list(goodreads_df.columns))
# Check number of columns
print(len(list(goodreads_df.columns)))

['user_id', 'book_id', 'review_id', 'rating', 'review_text', 'date_added', 'date_updated', 'read_at', 'started_at', 'n_votes', 'n_comments', 'isAmazon']
12


**Preprocessing** - Combine datasets

The combined dataset can have user ID, ASIN, Goodreads book ID, review text, and rating

In [ ]:
# Create a combined df
combined_df = pd.DataFrame()
# Create one column with the ASIN and another column with Goodreads book ID
# Amazon ASIN column converted to list
asin_col_list = amazon_df["asin"].tolist()
# List of zeros to be appended after the asin list,
# if assuming no Goodreads reviews have ASIN
zeros_asin_col_list = [0] * len(goodreads_df.index)
# Create combined list
combined_asin_col_list = asin_col_list + zeros_asin_col_list
# Assign list to column
combined_df["asin"] = combined_asin_col_list

# List of zeros to be appended before the book_id list,
# if assuming no Amazon reviews have Goodreads book ID
zeros_book_id_col_list = [0] * len(amazon_df.index)
# Goodreads book_id column converted to list
book_id_col_list = goodreads_df["book_id"].tolist()
# Create combined list
combined_book_id_col_list = zeros_book_id_col_list + book_id_col_list
# Assign list to column
combined_df["book_id"] = combined_book_id_col_list
print("Finished asin and book id concat")
print(combined_df.head())

# Create a column made up of Amazon and Goodreads user IDs
combined_df["user_id"] = amazon_df["reviewerID"] + goodreads_df["user_id"]
# Create a column made up of both Amazon and Goodreads review texts
print("Finished user id concat")
combined_df["review_text"] = amazon_df["reviewText"] + goodreads_df["review_text"]
print("Finished review text concat")
# Create a column made up of both Amazon ratings and Goodreads ratings
# Amazon reviews are doubles, Goodreads are ints
combined_df["rating"] = amazon_df["overall"] + goodreads_df["rating"]
print("Finished rating concat")
# Create a column saying whether the review was originally from Amazon or Goodreads
combined_df["isAmazon"] = amazon_df["isAmazon"] + goodreads_df["isAmazon"]
print("Finished is amazon concat")

combined_df.head()

Finished user id concat


In [ ]:
# Save the combined dataframe as a CSV
# combined_df.to_csv("combined_reviews_8mill.csv", index=False)
print("Finished saving.")

We can also check the review text of the combined dataset to make sure there are no duplicate review texts.

# **Training**

**Training** - Train a model on the Amazon dataset

In [ ]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

import numpy as np

In [ ]:
# mount to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Get the first 4 million reviews from CSV named amazon_reviews_1 in Google Drive

# Go to folder containing CSV
%cd "/content/drive/My Drive/data/ce256/project"
# load first 4 million reviews
amazon_df = pd.read_csv("amazon_reviews_1")

# Check the CSV in Google Drive has the correct data
amazon_df.head()

/content/drive/My Drive/data/ce256/project


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,False,"03 30, 2005",A1REUF3A1YCPHM,0001713353,{'Format:': ' Hardcover'},TW Ervin II,"The King, the Mice and the Cheese by Nancy Gur...",A story children will love and learn from,1112140800,NaN,NaN
1,5.0,True,"06 20, 2016",AVP0HXC9FG790,0001713353,NaN,Amazon Customer,The kids loved it!,Five Stars,1466380800,NaN,NaN
2,5.0,True,"01 24, 2016",A324TTUBKTN73A,0001713353,{'Format:': ' Paperback'},Tekla Borner,My students (3 & 4 year olds) loved this book!...,Five Stars,1453593600,NaN,NaN
3,5.0,False,"07 9, 2015",A2RE7WG349NV5D,0001713353,{'Format:': ' Paperback'},Deborah K Woroniecki,LOVE IT,Five Stars,1436400000,NaN,NaN
4,5.0,True,"01 18, 2015",A32B7QIUDQCD0E,0001713353,NaN,E,Great!,Five Stars,1421539200,NaN,NaN


In [ ]:
# Create ratings df containing only user ID, ASIN as book ID, and rating
'''
amazon_ratings_df = pd.DataFrame()
amazon_ratings_df["reviewerId"] = amazon_df["reviewerId"]
amazon_ratings_df["asin"] = amazon_df["asin"]
amazon_ratings_df["overall"] = amazon_df["overall"]
'''
amazon_ratings_df = amazon_df[["reviewerID", "asin", "overall"]].copy()
amazon_ratings_df.head()

,reviewerID,asin,overall
0,A1REUF3A1YCPHM,0001713353,5.0
1,AVP0HXC9FG790,0001713353,5.0
2,A324TTUBKTN73A,0001713353,5.0
3,A2RE7WG349NV5D,0001713353,5.0
4,A32B7QIUDQCD0E,0001713353,5.0


In [ ]:
# Save the Amazon user ID, ASIN as book ID, and rating dataframe as a CSV
# amazon_ratings_df.to_csv("ratings_4mill_amazon.csv", index=False)
# print("Finished saving.")

In [ ]:
# Credits-Prof Eirinaki, Rashmi Sharma and Aditya Patel
# conda install -c conda-forge scikit-surprise
!pip install scikit-surprise
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection.split import train_test_split
from surprise.model_selection import cross_validate, GridSearchCV
import pandas as pd
import numpy as np
import os, io
from surprise import KNNBasic, KNNWithMeans
from surprise import SVDpp
from surprise import SVD
from surprise import accuracy

     |████████████████████████████████| 11.8MB 365kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670933 sha256=2a75f33f5259eae36a5aeb8c5d758ed09331e410cb7f7f181b828814d53f5515
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
reader = Reader(rating_scale=(1,5))  #invoke reader instance of surprise library
data=Dataset.load_from_df(amazon_ratings_df,reader) #load dataset into Surprise datastructure Dataset

In [ ]:
#create training set
trainingSet, testSet = train_test_split(data, test_size=0.2, train_size=None, random_state=None, shuffle=True)

In [ ]:
#SVD
svd = SVD()
svd.fit(trainingSet) #fit model to the training set
predictions_svd = svd.test(testSet) #predict for test set values

Testing

In [ ]:
#validating rating predictions using RMSE
accuracy.rmse(predictions_svd, verbose=True) 

RMSE: 0.9682


0.9681906345336332

Making Predictions

In [ ]:
#print(predictions_svd[0])
# print(predictions_svd[1])
print(len(predictions_svd))
# Get predicted and actual ratings of a user
for pred in predictions_svd:
  if pred[0] == "AQEO3JYVJJH31":
    print(pred)

user: AV1LMQTN6YCIH item: 0307583260 r_ui = 2.00   est = 3.15   {'was_impossible': False}
800000
user: AQEO3JYVJJH31 item: 0091944244 r_ui = 5.00   est = 4.54   {'was_impossible': False}


In [ ]:
# Get predicted ratings of a user in the dataset
amazon_user_id = "AQEO3JYVJJH31"
svd.predict(amazon_user_id, "0091944244") # A Gentleman in Moscow, user already rated this 5.0
svd.predict(amazon_user_id, "0007548672") # All the Light We Cannot See, user already rated this 5.0
svd.predict(amazon_user_id, "0001713353") # The King, the Mice and the Cheese
svd.predict(amazon_user_id, "0001384198") # The Little Engine that Could
svd.predict(amazon_user_id, "0002005263") # The Sinister Pig
svd.predict(amazon_user_id, "059035342X") # Harry Potter and the Sorcerer's Stone, not in dataset

Prediction(uid='AQEO3JYVJJH31', iid='059035342X', r_ui=None, est=4.479512448921906, details={'was_impossible': False})

Making predictions using user and book dfs

In [ ]:
# read the csv into a dataframe
#path = "/content/drive/My Drive/data/ce256/project/sample_user_name.csv"
#user_df = pd.read_csv(path)

user_df = pd.read_csv("sample_user_name.csv")

In [ ]:
user_df.head(5)

,username,id
0,user1,AQEO3JYVJJH31


In [ ]:
user_dict = {}
for i in range(len(user_df)):
    user_dict[user_df.iloc[i].username] = user_df.iloc[i].id

In [ ]:
print(user_dict)

{'user1': 'AQEO3JYVJJH31'}


In [ ]:
# read the csv into a dataframe
path = "/content/drive/My Drive/data/ce256/project/sample_book_name.csv"
book_df = pd.read_csv(path)

#movie_df = pd.read_csv("movie_name.csv")

In [ ]:
book_df.head(5)

,bookName,id
0,A Gentleman in Moscow,91944244
1,All the Light We Cannot See,7548672
2,The Anatomy of Peace: Resolving the Heart of C...,141047666
3,The Complete Idiot's Guide to Music Theory,28643771


In [ ]:
"""Add leading zeros to get the correct 10-digit ASIN.
:param book_df: The df containing a column called "id" which need to be 10-digit ASINs
"""
def add_leading_zeros_to_ids(book_df):
  for id in book_df["id"]:
    # print(str(id).zfill(10))
    ten_digit_id = str(id).zfill(10)
    book_df["id"] = book_df["id"].replace([id], ten_digit_id)

In [ ]:
add_leading_zeros_to_ids(book_df)
book_df.head()

,bookName,id
0,A Gentleman in Moscow,0091944244
1,All the Light We Cannot See,0007548672
2,The Anatomy of Peace: Resolving the Heart of C...,0141047666
3,The Complete Idiot's Guide to Music Theory,0028643771


In [ ]:
book_dict = {}
for i in range(len(book_df)):
    book_dict[book_df.iloc[i].id] = book_df.iloc[i].bookName

In [ ]:
print(book_dict)

{'0091944244': 'A Gentleman in Moscow', '0007548672': 'All the Light We Cannot See', '0141047666': 'The Anatomy of Peace: Resolving the Heart of Conflict', '0028643771': "The Complete Idiot's Guide to Music Theory"}


In [ ]:
from collections import defaultdict

def getBookRecommendations(topN=3):
    top_recs = defaultdict(list)
    # for uid, iid, true_r, est, _ in predictions: 
    for uid, iid, true_r, est, _ in predictions_svd: 
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]
     
    return top_recs 

In [ ]:
recommendations = getBookRecommendations(3)

In [ ]:
def getBookName(book_id):
    if book_id not in book_dict:
        return ""
    b = book_dict[book_id]
    return b

In [ ]:
def getBookRecommendationsForUser(userId, recommendations):
    if userId not in user_dict:
        print("User id is not present")
        return
    u_id = user_dict[userId]
    recommended_books = recommendations[u_id]
    book_list = []
    for book in recommended_books:
        book_list.append((getBookName(book[0]),book[1]))
    return book_list    

In [ ]:
# change to user ID
getBookRecommendationsForUser('user1',recommendations)

[('A Gentleman in Moscow', 4.539754210244321)]

**Training** - Train a model on the Goodreads dataset

In [ ]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

import numpy as np

In [ ]:
# mount to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Get the first 4 million reviews from CSV named goodreads_reviews_1 in Google Drive

# Go to folder containing CSV
%cd "/content/drive/My Drive/data/ce256/project"
# load first 4 million reviews
goodreads_df = pd.read_csv("goodreads_reviews_1.csv")

# Check the CSV in Google Drive has the correct data
goodreads_df.head()

/content/drive/My Drive/data/ce256/project


,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,24375664,5cd416f3efc3f944fce4ce2db2290d5e,5,Mind blowingly cool. Best science fiction I've...,Fri Aug 25 13:55:02 -0700 2017,Mon Oct 09 08:55:59 -0700 2017,Sat Oct 07 00:00:00 -0700 2017,Sat Aug 26 00:00:00 -0700 2017,16,0
1,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
2,8842281e1d1347389f2ab93d60773d4d,6392944,5e212a62bced17b4dbe41150e5bb9037,3,I haven't read a fun mystery book in a while a...,Mon Jul 24 02:48:17 -0700 2017,Sun Jul 30 09:28:03 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,Mon Jul 24 00:00:00 -0700 2017,6,0
3,8842281e1d1347389f2ab93d60773d4d,22078596,fdd13cad0695656be99828cd75d6eb73,4,"Fun, fast paced, and disturbing tale of murder...",Mon Jul 24 02:33:09 -0700 2017,Sun Jul 30 10:23:54 -0700 2017,Sun Jul 30 15:42:05 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,22,4
4,8842281e1d1347389f2ab93d60773d4d,6644782,bd0df91c9d918c0e433b9ab3a9a5c451,4,A fun book that gives you a sense of living in...,Mon Jul 24 02:28:14 -0700 2017,Thu Aug 24 00:07:20 -0700 2017,Sat Aug 05 00:00:00 -0700 2017,Sun Jul 30 00:00:00 -0700 2017,8,0


In [ ]:
# Create ratings df containing only user ID, Goodreads book ID, and rating
goodreads_ratings_df = goodreads_df[["user_id", "book_id", "rating"]].copy()
goodreads_ratings_df.head()

,user_id,book_id,rating
0,8842281e1d1347389f2ab93d60773d4d,24375664,5
1,8842281e1d1347389f2ab93d60773d4d,18245960,5
2,8842281e1d1347389f2ab93d60773d4d,6392944,3
3,8842281e1d1347389f2ab93d60773d4d,22078596,4
4,8842281e1d1347389f2ab93d60773d4d,6644782,4


In [ ]:
# Save the Goodreads user ID, book ID, and rating dataframe as a CSV
# goodreads_ratings_df.to_csv("ratings_4mill_goodreads.csv", index=False)
# print("Finished saving.")

In [ ]:
# Credits-Prof Eirinaki, Rashmi Sharma and Aditya Patel
# conda install -c conda-forge scikit-surprise
!pip install scikit-surprise
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection.split import train_test_split
from surprise.model_selection import cross_validate, GridSearchCV
import pandas as pd
import numpy as np
import os, io
from surprise import KNNBasic, KNNWithMeans
from surprise import SVDpp
from surprise import SVD
from surprise import accuracy

     |████████████████████████████████| 11.8MB 4.9MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670934 sha256=300dfb6ac0e708174c143716697329490c95262794eb73925fca0fdfbd05acc2
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
reader = Reader(rating_scale=(1,5))  #invoke reader instance of surprise library
data=Dataset.load_from_df(goodreads_ratings_df,reader) #load dataset into Surprise datastructure Dataset

In [ ]:
#create training set
trainingSet, testSet = train_test_split(data, test_size=0.2, train_size=None, random_state=None, shuffle=True)

In [ ]:
#SVD
svd = SVD()
svd.fit(trainingSet) #fit model to the training set
predictions_svd = svd.test(testSet) #predict for test set values

Testing

In [ ]:
#validating rating predictions using RMSE
accuracy.rmse(predictions_svd, verbose=True) 

RMSE: 1.0788


1.0787506691890305

Making Predictions

In [ ]:
# Get predicted ratings of a user in the dataset
goodreads_user_id = "8842281e1d1347389f2ab93d60773d4d"
svd.predict(goodreads_user_id, "24375664") # The Dark Forest (Remembrance of Earth’s Past, #2), user already rated this 5.0
svd.predict(goodreads_user_id, "18245960") # The Three-Body Problem (Remembrance of Earth’s Past, #1), user already rated this 5.0
svd.predict(goodreads_user_id, "6392944") # The Murder on the Links (Hercule Poirot, #2), user already rated this 3.0

Prediction(uid='8842281e1d1347389f2ab93d60773d4d', iid='6392944', r_ui=None, est=3.3759328843860725, details={'was_impossible': False})

In [ ]:
# Get the titles of these books using client and Goodreads book ID
import time
get_title_with_goodreads_api(client, "24375664")
time.sleep(1)
get_title_with_goodreads_api(client, "18245960")
time.sleep(1)
get_title_with_goodreads_api(client, "6392944")

{'id': '24375664', 'title': 'The Dark Forest (Remembrance of Earth’s Past, #2)', 'isbn': None}
{'id': '18245960', 'title': 'The Three-Body Problem (Remembrance of Earth’s Past, #1)', 'isbn': None}
{'id': '6392944', 'title': 'The Murder on the Links (Hercule Poirot, #2)', 'isbn': None}


{'id': '6392944',
 'isbn': None,
 'title': 'The Murder on the Links (Hercule Poirot, #2)'}

---
**Display** title and authors

After getting final list of recommended books of an Amazon user, display title and authors.

The model should know which users and books are unique. This is to make sure the model is not accidentally recommending the same book from Goodreads that a user has reviewed on Amazon.

The model will assume all users on Amazon are 
different from all users on Goodreads.
That is, all user IDs are unique. There is no user with both an Amazon and Goodreads ID.

To know which books are unique, we can check the book title and authors. If two books have the same title and authors, they are the same book.

- The Amazon ASIN of a book is the same as the book's 10-digit ISBN (International Standard Book Number). 
- Can use Google Books API and ISBN to get book title and author.
 
See links:
- https://www.oreilly.com/library/view/amazon-hacks/0596005423/ch01s03.html
- https://gist.github.com/AO8/faa3f52d3d5eac63820cfa7ec2b24aa7

In [ ]:
"""Searches a dictionary and returns title and authors if they exist,
returns None otherwise.

:param obj: a Python dictionary that was decoded from JSON
:param isbn: the ISBN this book should have
:returns: The result's title and authors if title and authors can be found, 
None otherwise
"""
def get_title_and_authors_by_isbn(obj, isbn):
    volume_info = obj["items"][0] 
    if "title" in volume_info["volumeInfo"] and "authors" in volume_info["volumeInfo"]:
      title = volume_info["volumeInfo"]["title"]
      authors = volume_info["volumeInfo"]["authors"]
      return (title, authors)
    else:
      print("Title or authors not found for ISBN: " + isbn)
      return None

In [ ]:
"""Uses Google Books API and an ISBN 
and returns title and author of book with that ISBN.

:param isbn: the ISBN this book should have
:returns: The title and author if ISBN, title, and author can be found, None otherwise
"""
def get_title_and_authors_with_books_api(isbn):
  with urlopen(base_api_link + curr_asin) as f:
          text = f.read()
  decoded_text = text.decode("utf-8")
  # deserializes decoded_text to a Python object (dictionary)
  obj = json.loads(decoded_text)
  # pretty_print_json(obj)
  if ("items" in obj):
    # print("current ISBN: " + curr_asin)
    return get_title_and_authors_by_isbn(obj, curr_asin)
  else:
    print("No items found with ISBN: " + curr_asin)
    return None

In [ ]:
"""Print a dictionary decoded from JSON into a human-readable format, with 4 tabs
:param obj: a Python dictionary that was decoded from JSON
:returns: A human-readable String
"""
def pretty_print_json(obj):
  readable_json_str = json.dumps(obj, indent=4, sort_keys=True)
  print(readable_json_str)
  return readable_json_str

In [ ]:
# Get ASIN of all books as a list
asin_list = amazon_df["asin"].tolist()

# Get ASIN list without duplicate ASINs
no_duplicates_asin_list = list( dict.fromkeys(asin_list) )
# print length to get number of unique ASINs
print(len(no_duplicates_asin_list))

# List of all titles
title_list = []
# List of all authors
author_list = []

# Use Google Books API and ASIN to get title and authors.
# Too many requests to API, ex. 1000 requests, gives HTTPError: HTTP Error 429: Too Many Requests
# Using isbn: or ISBN: in search query may not return any items at all
base_api_link = "https://www.googleapis.com/books/v1/volumes?q=isbn:"
# Index of current ASIN
asin_index = 0
# Loop through all unique ASINs and get each book's title and authors
while asin_index < 100:
  curr_asin = no_duplicates_asin_list[asin_index]

  # Result is a tuple of two elements. Element at index 0 is the title, and
  # element at index 1 is the list of authors.
  result_title_and_authors = get_title_and_authors_with_books_api(curr_asin)
  if (result_title_and_authors != None):
    # append title
    title_list.append(result_title_and_authors[0])
    # append authors
    author_list.append(result_title_and_authors[1])
  asin_index += 1

# Use ASIN of each review to get the book's title and authors
for title in title_list:
  print(title)
for authors in author_list:
  # print each element from author list, separated by comma and space
  print(", ".join(authors))

- Goodreads book ID can be used to get title and author using separate books metadata dataset or Goodreads API.
See links:
  - https://pypi.org/project/Goodreads/
  - https://www.goodreads.com/api
  - https://sites.google.com/eng.ucsd.edu/ucsdbookgraph/books


In [ ]:
"""Uses Goodreads API Python client and a Goodreads ID
and returns Goodreads ID, title, and ISBN of book with that ID.

:param client: The Goodreads API Python client
:param id: the Goodreads ID this book should have
:returns: Dictionary with the Goodreads ID, title, and ISBN of the book
"""
def get_title_with_goodreads_api(client, id):
  book = client.Book.show(id)
  keys_wanted = ['id', 'title', 'isbn']
  reduced_book = {k:v for k, v in book.items() if k in keys_wanted}
  print(reduced_book)
  return reduced_book

In [ ]:
"""Uses Goodreads API Python client and an ISBN
and returns Goodreads ID, title, and ISBN of book with that ID.

:param client: The Goodreads API Python client
:param isbn: The ISBN this book should have
:returns: Dictionary with the Goodreads ID, title, and ISBN of the book
"""
def get_title_with_goodreads_api_and_isbn(client, isbn):
  book = client.Book.show_by_isbn(isbn)
  keys_wanted = ['id', 'title', 'isbn']
  reduced_book = {k:v for k, v in book.items() if k in keys_wanted}
  print(reduced_book)
  return reduced_book

In [ ]:
# Test using Goodreads API on a random Goodreads book ID
get_title_with_goodreads_api(client, "1")
get_title_with_goodreads_api(client, "4986701")
# Test using Goodreads API on a random ISBN
get_title_with_goodreads_api_and_isbn(client, "0001932349")
get_title_with_goodreads_api_and_isbn(client, "0002005263")

{'id': '1', 'title': 'Harry Potter and the Half-Blood Prince (Harry Potter, #6)', 'isbn': None}
{'id': '4986701', 'title': 'Ready-to-Use Old-Fashioned Illustrations of Books, Reading and Writing', 'isbn': '0486270939'}
{'id': '1971963', 'title': 'Love is a Special Way of Feeling', 'isbn': '0001932349'}
{'id': '9874253', 'title': 'The Sinister Pig', 'isbn': '0002005263'}


{'id': '9874253', 'isbn': '0002005263', 'title': 'The Sinister Pig'}